In [17]:
import tensorflow as tf
import os.path
import numpy as np
import abc
import time
import datetime as dt
from neuralimg.dataio import *
from six.moves import xrange  # pylint: disable=redefined-builtin

In [18]:
def create_conv_weights(i, ksize, inps, outs, m=0.0, stddev=1e-1, bias=0):
    """ Creates convolutional weights and bias. Initializes the first with a
        truncated normal distribution and the second with a constant
        :param i: Identifier of the weights
        :param ksize: Size of the kernel
        :param outs: Depth of the output
        :param m: Mean of the initialization of the weights
        :param stddev: Standard deviation of the weights initialization
        :param bias: Initialization value for the bias
    """
    w = tf.get_variable('convw' + str(i), [ksize, ksize, inps, outs],
            initializer = tf.truncated_normal_initializer(mean=m, stddev=stddev))
    
    b = tf.get_variable('convb' + str(i) , [outs], 
            initializer = tf.constant_initializer(bias))
    return w, b

def create_fc_weights(i, inps, hiddens, m=0.0, stddev=1e-1, bias=0):
    """ Creates convolutional weights and bias. Initializes the first with a
        truncated normal distribution and the second with a constant
        :param i: Identifier of the weights
        :param inps: Size of the inputs
        :param hiddens: Number of hidden units
        :param m: Mean of the initialization of the weights
        :param stddev: Standard deviation of the weights initialization
        :param bias: Initialization value for the bias
    """
    w = tf.get_variable('fullyw' + str(i), [inps, hiddens],
            initializer = tf.truncated_normal_initializer(mean=m, stddev=stddev))
    
    b = tf.get_variable('fullyb' + str(i) , [hiddens], 
            initializer = tf.constant_initializer(bias))
    return w, b

def summary_activations(name, x):
    """ Defines a histogram and a sparsity measure of the activations for the input function """
    tf.histogram_summary(name + '/activations', x)
    tf.scalar_summary(name + '/sparsity', tf.nn.zero_fraction(x))

def define_convolution(data, w, b, i, maxp, nr, prefix):
    """ Returns a convolutional layer with the input parameters. Weights are initialized as truncated
    normals around 0 and stdev of 1e-3 and bias as 0.
        :param data: Input placeholder
        :param i: Identifier of the convolutional layer
        :param maxp: Whether to perform max pooling after the activation. TODO: check characteristics of max pooling
        :param nr: Whether to perform loal response normalisation at the end TODO: check charcateristics of this
        :param prefix: Prefix of the layer (left/right/center)
    """
    
    # Convolution operation
    conv = tf.nn.conv2d(data, w, strides=[1, 1, 1, 1], padding='SAME', 
                         name=prefix + '-conv' + str(i))
    
    # Perform ReLU(x) = max(0, x)
    relu = tf.nn.relu(tf.nn.bias_add(conv, b), name=prefix + '-relu' + str(i))
    
    # Perform max pooling, if requested
    if maxp is True:
        relu = tf.nn.max_pool(relu, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], 
                               padding='SAME', name=prefix + '-pool' + str(i))
    
    # Perform Local Response Normalisation if requested
    if nr is True:
        relu = tf.nn.lrn(relu, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name=prefix + '-norm' + str(i))  
    
    # Add summary for all operations
    summary_activations(prefix + '-conv' + str(i), relu)
    
    return relu


def define_fully_layer(data, w, b, i, prefix, dropout=None):
    """ Returns a fully connected layer with the input parameters. Weights are initialized as truncated
    normals around 0 and stdev of 1e-3 and bias as 0.
        :param data: Input placeholder
        :param i: Identifier of the convolutional layer
        :param prefix: Prefix of the layer (left/right/center)
        :param mode: Mode to use (training, validation, testing)
        :param dropout: Keep probability of the dropout (in interval [0,1]). None if not used
        # TODO: use different activation functions?
    """
    
    # Perform ReLU(x) = max(0, x)
    relu = tf.nn.relu(tf.nn.bias_add(tf.matmul(data, w), b), name=prefix + '-relufc' + str(i))
    
    # Add dropout if requested
    if dropout is not None:
        relu = tf.nn.dropout(relu, dropout)

    summary_activations(prefix + '-relu' + str(i), relu)
        
    return relu

def store_checkpoint(session, saver, step, path, name='model'):
    """ Stores a checkpoint of the model
        :param session: Tensorflow session
        :param saver: Tensorflow saver
        :param step: Training step where the model belongs to
        :param path: Folder where to store the model
        :param name: Name of the checkpoint file
    """
    checkpoint_path = os.path.join(path, name + '_' + str(step) + '.ckpt')
    saver.save(session, checkpoint_path, global_step=step)
        
def euclidean_dist(f1, f2):
    """ Computes the euclidean distance between two tensors """
    # Must be float, convert to be sure
    f1 = tf.cast(f1, tf.float32)
    f2 = tf.cast(f2, tf.float32)
        
    # Compute euclidean distance between outputs
    subs = tf.sub(f1, f2)       
    pows = tf.pow(subs, tf.constant(2, tf.float32, shape=subs.get_shape()))
    suma = tf.reduce_sum(pows, 1)
    return  tf.sqrt(suma)

In [19]:
# TODO: options can be parsed from a file

class SiameseNetwork(object):
    
    __metaclass__ = abc.ABCMeta
    
    def __init__(self, opts):
        self.opts = opts
        self.scope_name = 'siamese'
        self.loss_interval = 10
        self.summary_interval = 2
        self.checkp_interval = 1000
        self.train_loss = []
        self.val_loss = []
        self.check_options()
        self.val_track = []
        
    @abc.abstractmethod
    def get_placeholder_suffixes(self):
        "Returns the placeholders according to the subclass "
    
    @abc.abstractmethod
    def get_loss(self, placeholders):
        " Defines the specific loss function to use"
    
    @abc.abstractmethod
    def get_label_batch(self, offset, batch_size):
        " Returns the labels for the corresponding offset "
    
    
    def check_options(self):
        """ Parser the options """
        self.keep_prob = self.opts['dropout'] if 'dropout' in self.opts else None
        self.early = self.opts['early'] if 'early' in self.opts else None
        self.track_val = [] if 'early' in self.opts else None
        
        
    def read_data(self, path):
        """ Reads the data from the input dataset
            :param path: Path to the HDF5 containing the data. Must have a group for
                each set: validation, training and testing with datasets 'data' and 'labels'
        """
        dr = DatasetReader(path)
        data = dr.read()
        self.tr_data = reshape_data(data['training']['data'])
        self.tr_labels = data['training']['labels']
        self.val_data = reshape_data(data['validation']['data'])
        self.val_labels = data['validation']['labels']
        self.test_data = reshape_data(data['testing']['data'])
        self.test_labels = data['testing']['labels']
        self.insts, self.imh, self.imw, self.depth = self.tr_data.shape[1:]
        
        
    def _define_weights(self):
        """ Defines the weights needed in he network """
        # TODO: do it in an automatic way
        wc1, bc1 = create_conv_weights(1, self.opts['ksize'], 3, self.opts['maps'])
        wc2, bc2 = create_conv_weights(2, self.opts['ksize'], self.opts['maps'], self.opts['maps'])
        wc3, bc3 = create_conv_weights(3, self.opts['ksize'], self.opts['maps'], self.opts['maps'])
        
        # Number dividing depends on number of convolutions
        wf1, bf1 = create_fc_weights(1, self.imh // 8 * self.imw // 8 * self.opts['maps'], self.opts['hiddens1'])
        wf2, bf2 = create_fc_weights(2, self.opts['hiddens1'], self.opts['hiddens2'])
        return [[wc1, bc1], [wc2, bc2], [wc3, bc3], [wf1, bf1], [wf2, bf2]]
    
    
    def model(self, images, prefix):
        """ Defines the graph model 
            :param images: Input data
            :param prefix: Prefix to add to each operator to identify elemtn in pair/trilet
        """
        
        # Obtain weights (already created)
        ws = self._define_weights()
        
        tf.image_summary(prefix, images, max_images=3)
        
        # Convolutional layers
        with tf.variable_scope("conv1_" + prefix) as scope:
            norm1 = define_convolution(images, ws[0][0], ws[0][1], 1, 
                                       self.opts['maxp'], self.opts['lrn'], prefix)
            
        with tf.variable_scope("conv2_" + prefix) as scope:
            norm2 = define_convolution(norm1,  ws[1][0], ws[1][1], 2, 
                                       self.opts['maxp'], self.opts['lrn'], prefix)
                                                                            
        with tf.variable_scope("conv3_" + prefix) as scope:
            norm3 = define_convolution(norm2,  ws[2][0], ws[2][1], 3, 
                                       self.opts['maxp'], self.opts['lrn'], prefix)
        
        # Flatten convolutions output
        shape = norm3.get_shape().as_list()   
        flat = tf.reshape(norm3, [shape[0], shape[1] * shape[2] * shape[3]])
        
        # Fully connected layers
        with tf.variable_scope("fully1_" + prefix) as scope:
            fc1 = define_fully_layer(flat,  ws[3][0], ws[3][1], 1, prefix, self.keep_prob)
        
        with tf.variable_scope("fully2_" + prefix) as scope:
            fc2 = define_fully_layer(fc1,  ws[4][0], ws[4][1], 2, prefix, self.keep_prob)
        
        # Store the variables for further examination
        self.v = [norm1, norm2, norm3, fc1, fc2]
        
        return fc2
    
    def define_placeholders(self):
        """ Initializes all placeholders needed in the networ """
        self.pls = self.get_data_placeholders() # Data
        self.pl_labels = tf.placeholder(tf.float32, shape=(self.opts['batch_size'])) # Labels
        self.keep_prob = tf.placeholder(tf.float32, name='keep_probability') # Dropout
    
    def get_data_placeholders(self):
        """ Returns the input placeholders/constants for the data """
        names = self.get_placeholder_suffixes()
        training = [tf.placeholder(tf.float32, shape=(self.opts['batch_size'], self.imh, self.imw, 
                                                 self.depth), name=i) for i in names]
        return training
    
    def evaluate_net(self, data, labels, session):     
        """ Computes the mean loss for the input setwithout upgrading the weights
            Args:
                data: Input image data
                labels: Input label data
                session: Current execution session
            Returns:
                Loss and duration of the evaluation
        """
        size = data.shape[0]
        if size < self.opts['batch_size']:
            raise ValueError("Batch size for evals larger than dataset: %d" % size)
        
        losses = []
        start_time = time.time()
        for begin in xrange(0, size, self.opts['batch_size']):
            
            # Obtain data for current step
            end = begin + self.opts['batch_size']
            d = data[begin:end, ...] if end <= size else data[-self.opts['batch_size']:, ...]
            l = labels[begin:end] if end <= size else labels[-self.opts['batch_size']:]
            
            # Feed into the network to obtain the loss
            loss_val = session.run(self.loss, feed_dict=self.build_feed(d, l))
            losses.append(loss_val)
            
        duration = time.time() - start_time
        return numpy.mean(losses), duration

    def build_feed(self, batch_data, batch_labels):
        """ Builds a dictionary that feeds the network"""
        feed = {pl: batch_data[:, i, ...] for i, pl in enumerate(self.pls)}
        feed.update({self.pl_labels: batch_labels, self.keep_prob: self.opts['dropout']})
        return feed
    
    def perform_training(self, batch_data, batch_labels, session, step):
        """Performs a training operation and returns the loss and the duration taken.
            Args:
                batch_data: Input data batch
                batch_labels: Input label batch
                session: Current execution session
                step: Step at which we want to store a summary. None to disable storing
            Returns:
                Loss and duration of the training
        """
        start_time = time.time()
        feed = self.build_feed(batch_data, batch_labels)
        if step is None:
            print('yes')
            s, l = session.run([self.optimizer, self.loss], feed_dict=feed)
            duration = time.time() - start_time
        else:
            print('no')
            summary_str, r, l = session.run([self.summary_op, self.optimizer, self.loss], feed_dict=feed)
            duration = time.time() - start_time
            self.sum_writer.add_summary(summary_str, step)
        return l, duration
    
    def get_batch(self, step):
        """ Returns a batch from the training data according to the current step"""
        offset = (step * self.opts['batch_size']) % (self.tr_data.shape[0] - self.opts['batch_size'])
        batch_data = self.tr_data[offset:(offset + self.opts['batch_size']), ...]
        batch_labels = self.get_label_batch(offset, self.opts['batch_size'])
        return batch_data, batch_labels
    
    def track_validation(self, step, new_value, session, saver, outp):
        """ Tracks the window of validation losses and checks whether the new value """
        if len(self.track_val) < self.early:
            self.track_val.append(new_value)
            return False
        if self.track_val[0] < new_value:
            return True
        else:
            self.track_val = self.track[1:] + [new_value]
            store_checkpoint(session, saver, step, outp, 'early')
            return False
    
    def train(self, outp):
        """ Buils the network given the read data and starts training
            :param out: Where to store the resulting model and the checkpoints
        """
        
        # Restore from past checkpoint - TODO
        
        dev = 'gpu' if self.opts['gpu'] is True else 'cpu'
        g = tf.Graph()

        with g.as_default(), tf.device('/' + dev + ':0'):
            
            with tf.variable_scope(self.scope_name) as scope:
                
                # Create shared weights in the root scope
                ws = self._define_weights()
                scope.reuse_variables()
                
                self.define_placeholders()
                
                # Define losses and set gradient descent to minimize it
                self.loss = self.get_loss(self.pls, self.pl_labels)
                self.optimizer = tf.train.GradientDescentOptimizer(self.opts['lr']).minimize(self.loss)
    
                # Define summaries for all variables
                for var in tf.trainable_variables():
                    tf.histogram_summary(var.op.name, var)
    
            with tf.Session(graph=g, config=tf.ConfigProto(log_device_placement=True)) as session:

                # Initialize all defined variables
                tf.initialize_all_variables().run()
                
                # Prepare summaries for Tensorboard visualization
                self.summary_op = tf.merge_all_summaries()
                self.sum_writer = tf.train.SummaryWriter(self.opts['logs'], session.graph)             
                saver = tf.train.Saver()

                for step in range(1, self.opts['steps'] + 1):
                    self._training_step(session, step, saver, outp)
                
                store_checkpoint(session, saver, step, outp, name='final')
                
            
    def _training_step(self, session, step, saver, outp):
        """ Performs a single training step """

        tr_data, tr_labels = self.get_batch(step)

        # Update gradients. Each 5 steps (default, TODO change to 100) also write summary of operations
        s = None if step % self.summary_interval != 0 else step
        loss_value, duration = self.perform_training(tr_data, tr_labels, session, s)
        
        # Print loss each 10 steps (default)
        if step % self.loss_interval == 0:

            # Validation - Check loss evolution without updating gradients
            lv, durationv = self.evaluate_net(self.val_data, self.val_labels, session)
            
            assert not np.isnan(lv)

            # Print and save losses
            self.print_loss(loss_value, step, 'training', duration)
            self.print_loss(lv, step, 'validation', durationv)
            self.train_loss.append(loss_value)
            self.val_loss.append(lv)
            
            if self.track_val is not None:
                print('Early stopping checking ...')
                if self.track_validation(step, lv, session, saver, outp) is True:
                    print('Early stopping: Validation loss has not improved' + 
                      ' for {} steps. Last best model has been stored'.format(self.early))
                    return

        # Save checkpoint of the model each 1000 steps (default)
        if step % self.checkp_interval == 0:
            store_checkpoint(session, saver, step, outp)
        
        
    def print_loss(self, loss_value, step, label, duration):
        """ Prints loss according at given step"""
        examples_per_sec = self.opts['batch_size'] / duration
        format_str = ('%s: step %d, %s loss = %.2f (%.1f examples/sec; %.3f '
            'sec/batch)')
        print (format_str % (dt.datetime.now(), step, label, loss_value,
            examples_per_sec, float(duration)))

    def display_loss(self):
        """ Prints the training and the validation evolution """
        x = np.arange(0, len(self.train_loss)) * self.loss_interval
        t = plt.plot(x, self.train_loss, label='training')
        v = plt.plot(x, self.val_loss, label='validation')
        plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
        plt.xlabel('Step')
        plt.ylabel('Loss')
        plt.show()

            
class PairedSiamese(SiameseNetwork):
    
    
    def get_label_batch(self, offset, batch_size):
        return self.tr_labels[offset:(offset + batch_size)]
    
    def get_placeholder_suffixes(self):
        return ['left', 'right']
    
    def get_loss(self, placeholders, labels):
        left, right = placeholders
        logits1 = self.model(left, 'left')
        logits2 = self.model(right, 'right')
        return self._paired_loss(logits1, logits2, labels)
    
    def _paired_loss(self, f1, f2, labels, thao = 1):
        """ Loss for a evaluation of a pair of (non)-corresponding slices
            Paired loss L:
                L = 0.5 * d(f1, f2) if positive example
                L = 0.5 * max(0 , thao - d(f1, f2)) if negative example
            Where thao is usually 1
        """
        
        # Must be float, convert to be sure
        dists = euclidean_dist(f1, f2)
        
        # Masks of classes
        int_labels = tf.cast(labels, tf.int32)
        pos_ind = tf.cast(labels, tf.float32)
        neg_ind = tf.cast(tf.sub(tf.ones(labels.get_shape(), tf.int32), int_labels), tf.float32)
        
        # Apply positive and negative to both classes
        all_pos = dists
        all_neg = tf.sub(tf.constant([thao], shape=[dists.get_shape()[0]], dtype=dists.dtype), dists)
        all_neg = tf.maximum(tf.zeros(all_neg.get_shape(), tf.float32), all_neg)

        # Apply binary mask to both so set 0's where not correct
        binary_pos = tf.mul(pos_ind, all_pos)
        binary_neg = tf.mul(neg_ind, all_neg)
        
        # Sum both and multiply by 1/2
        summed = tf.add(binary_pos, binary_neg)
        summed = tf.mul(summed, tf.constant([0.5], shape=[summed.get_shape()[0]]))
        return tf.reduce_mean(summed)


class TripletSiamese(SiameseNetwork):
    
    def get_label_batch(self, offset, batch_size):
        # Return whatever since it is not used
        return np.empty((batch_size))
    
    def get_placeholder_suffixes(self):
        return ['left', 'center' 'right']
    
    def get_loss(self, placeholders, labels):
        left, center, right = placeholders
        logits1 = self.model(left, 'left')
        logits2 = self.model(center, 'center')
        logits3 = self.model(right, 'right')
        return self._triplet_loss(logits1, logits2, logits3)
    
    def _triplet_loss(self, f1, f2, f3, alpha = 1):
        """ Loss for a evaluation of a pair of (non)-corresponding slices
            Triplet loss L:
                L= 0.5 * max(0, d(f1, f2) - d(f1, f3) + alpha)
            Where alpha is usually 1
        """
        
        # Distances negative and positive
        dist_pos = euclidean_dist(f1, f2)
        dist_neg = euclidean_dist(f1, f3)

        # Get maximum 
        subs = tf.sub(dist_pos, dist_neg)
        added = tf.add(subs, tf.constant([alpha], shape=[subs.get_shape()[0]], dtype=subs.dtype))
        loss = tf.maximum(tf.zeros(added.get_shape(), added.dtype), added)
        
        # Multiply by 1/2
        const = tf.constant([0.5], shape=[loss.get_shape()[0]])
        weighted = tf.mul(loss, const)
        return tf.reduce_mean(weighted)


def reshape_data(x):
    """ Converts the data into the Tensorflow format with float32 type"""
    return np.moveaxis(x, 2, 4).astype(np.float32)

In [20]:
opts = {'batch_size': 128, 'ksize': 7, 'maps':25, 'maxp': True, 'lrn': False, 'lr': 1e-4,
        'steps': 10, 'logs': "/home/morad/tf/logs", 'dropout': 1, 'hiddens1': 100, 'hiddens2': 100,
       'early': 5, 'gpu': False}
siamese = PairedSiamese(opts)
#siamese.read_data('/DataDisk/morad/cremi/datasets/dataset.h5')
siamese.read_data('/home/morad/projects/neural/neuralimg/tests/out.h5')
siamese.train('/home/morad/tf/models')

yes
no
yes
no
yes
no
yes
no
yes
no
2016-07-08 13:11:10.323486: step 10, training loss = 3.06 (53.9 examples/sec; 2.375 sec/batch)
2016-07-08 13:11:10.323630: step 10, validation loss = 3.14 (40.6 examples/sec; 3.150 sec/batch)
Early stopping checking ...
